<a href="https://colab.research.google.com/github/jcsnuesi/food_sales_predictions/blob/main/Proyecto_1_Parte_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('content/')

Mounted at content/


In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn import set_config
set_config(display='diagram')

In [26]:
df = pd.read_csv('content/MyDrive/Colab Notebooks/sales_predictions.csv')
df

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.930,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052
...,...,...,...,...,...,...,...,...,...,...,...,...
8518,FDF22,6.865,Low Fat,0.056783,Snack Foods,214.5218,OUT013,1987,High,Tier 3,Supermarket Type1,2778.3834
8519,FDS36,8.380,Regular,0.046982,Baking Goods,108.1570,OUT045,2002,NaN,Tier 2,Supermarket Type1,549.2850
8520,NCJ29,10.600,Low Fat,0.035186,Health and Hygiene,85.1224,OUT035,2004,Small,Tier 2,Supermarket Type1,1193.1136
8521,FDN46,7.210,Regular,0.145221,Snack Foods,103.1332,OUT018,2009,Medium,Tier 3,Supermarket Type2,1845.5976


In [28]:
# Categorical - Ordinal columns
outlet_type_dict = {"Grocery Store":0, "Supermarket Type1":1,"Supermarket Type2":2,"Supermarket Type3":3}
df.Outlet_Type = df.Outlet_Type.replace(outlet_type_dict)
df['Outlet_Type'].value_counts()

1    5577
0    1083
3     935
2     928
Name: Outlet_Type, dtype: int64

In [31]:
#  Categorical - Ordinal columns
item_fat_content_dict = {"Low Fat":0,"LF":0,"low fat":0,"Regular":1,"reg":1}
df.Item_Fat_Content = df.Item_Fat_Content.replace(item_fat_content_dict)
df['Item_Fat_Content'].value_counts()

0    5517
1    3006
Name: Item_Fat_Content, dtype: int64

In [34]:
#  Categorical - Ordinal columns
outlet_size_dict = {"Small":0,"Medium":1,"High":2}
df.Outlet_Size = df.Outlet_Size.replace(outlet_size_dict)
df['Outlet_Size'].value_counts()

1.0    2793
0.0    2388
2.0     932
Name: Outlet_Size, dtype: int64

In [36]:
#  Categorical - Ordinal columns

outlet_identifier_dict = {"OUT010":0,"OUT013":1,"OUT017":2,"OUT018":3,"OUT019":4,"OUT027":5,"OUT035":6,"OUT045":7,"OUT046":8,"OUT049":9}
df.Outlet_Identifier = df.Outlet_Identifier.replace(outlet_identifier_dict)
df['Outlet_Identifier'].value_counts()

5    935
1    932
9    930
8    930
6    930
7    929
3    928
2    926
0    555
4    528
Name: Outlet_Identifier, dtype: int64

In [41]:
df.Outlet_Location_Type.value_counts()

Tier 3    3350
Tier 2    2785
Tier 1    2388
Name: Outlet_Location_Type, dtype: int64

In [43]:
#  Categorical - Ordinal columns
outlet_location_type_dict = {"Tier 1":0,"Tier 2":1,"Tier 3":2}
df.Outlet_Location_Type = df.Outlet_Location_Type .replace(outlet_location_type_dict)
df.Outlet_Location_Type.value_counts()

2    3350
1    2785
0    2388
Name: Outlet_Location_Type, dtype: int64

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   int64  
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   int64  
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   float64
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   int64  
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(5), int64(4), object(3)
memory usage: 799.2+ KB


In [38]:
print(df.isna().sum().sum(), 'Missing values')

3873 Missing values


In [44]:
X = df.drop(columns='Item_Outlet_Sales')
y = df['Item_Outlet_Sales']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [46]:
X_train.describe().round(0)

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
count,5285.0,6392.0,6392.0,6392.0,6392.0,6392.0,4580.0,6392.0,6392.0
mean,13.0,0.0,0.0,142.0,5.0,1998.0,1.0,1.0,1.0
std,5.0,0.0,0.0,63.0,3.0,8.0,1.0,1.0,1.0
min,5.0,0.0,0.0,31.0,0.0,1985.0,0.0,0.0,0.0
25%,9.0,0.0,0.0,94.0,2.0,1987.0,0.0,0.0,1.0
50%,13.0,0.0,0.0,144.0,5.0,1999.0,1.0,1.0,1.0
75%,17.0,1.0,0.0,187.0,7.0,2004.0,1.0,2.0,1.0
max,21.0,1.0,0.0,267.0,9.0,2009.0,2.0,2.0,3.0


In [47]:
num_selector = make_column_selector(dtype_include='number')
cat_selector =  make_column_selector(dtype_include='object')

num_cols = num_selector(X_train)
cat_cols = cat_selector(X_test)


print('numeric cols are', num_cols)
print('categorical cols are', cat_cols)

numeric cols are ['Item_Weight', 'Item_Fat_Content', 'Item_Visibility', 'Item_MRP', 'Outlet_Identifier', 'Outlet_Establishment_Year', 'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type']
categorical cols are ['Item_Identifier', 'Item_Type']


In [48]:
# Imputer
mean_imputer = SimpleImputer(strategy='mean')
freq_imputer = SimpleImputer(strategy='most_frequent')

# Transformers

ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')
scarler = StandardScaler()

In [49]:
numeric_pipeline = make_pipeline(mean_imputer, scarler)
numeric_pipeline

Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('standardscaler', StandardScaler())])

In [50]:
cat_pipeline =  make_pipeline(freq_imputer, ohe)
cat_pipeline

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                ('onehotencoder',
                 OneHotEncoder(handle_unknown='ignore', sparse=False))])

In [51]:
# Tuples to transform columns
numeric_tuple = (numeric_pipeline, num_selector)
cat_tuple =  (cat_pipeline, cat_selector)

preprocessor =  make_column_transformer(numeric_tuple,cat_tuple)
preprocessor

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f9bfbc59790>),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f9bfbc59460>)])

In [52]:
preprocessor.fit(X_train)

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f9bfbc59790>),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f9bfbc59460>)])

In [53]:
X_train_processed = preprocessor.transform(X_train)
X_test_processed = preprocessor.transform(X_test)


In [57]:
print(np.isnan(X_train_processed).sum(),'Missing values in training data')
print(np.isnan(X_test_processed).sum().sum(), 'missing values in testing data')

0 Missing values in training data
0 missing values in testing data
